In [1]:
# Data Handling
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error

# Common ML models
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

# Advanced Libraries
# import xgboost as xgb
# import lightgbm as lgb
# import catboost as cb

# Deep Learning (TensorFlow/Keras)
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Ignore Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import glob
import os
import random

# Set the directory where the machine CSV files are located
directory_path = "AlibabaDataPrep/processedData/"

# Get all CSV files in the directory matching the pattern
all_files = glob.glob(os.path.join(directory_path, "machine_*_processed.csv"))

# Pick a random sample of 500 files
random_files = random.sample(all_files, 500)

print(len(random_files))


500


In [6]:
from tqdm import tqdm  # Import tqdm

# Read and concatenate all CSVs into one DataFrame with progress bar
df_list = [pd.read_csv(file) for file in tqdm(random_files, desc="Reading CSVs")]

df = pd.concat(df_list, ignore_index=True)

# Optional: preview merged data
print(f"Merged {len(df_list)} files. Final shape: {df.shape}")
df.head()


Reading CSVs: 100%|██████████| 500/500 [00:27<00:00, 17.93it/s]


Merged 500 files. Final shape: (22919107, 11)


,timestamp,index,machine_id,cpu_util_percent,mem_util_percent,mem_gps,mkpi,net_in,net_out,disk_io_percent,cpu_difference
0,1970-01-01 00:00:00,0.000000,0.0,0.010638,0.48,0.284916,1.0,0.0,0.0,0.012195,NaN
1,1970-01-01 00:00:15,0.000016,0.0,0.031915,0.48,0.284916,1.0,0.0,0.0,0.012195,0.405530
2,1970-01-01 00:00:30,0.000040,0.0,0.042553,0.48,0.284916,1.0,0.0,0.0,0.012195,0.396313
3,1970-01-01 00:00:45,0.000064,0.0,0.053191,0.48,0.284916,1.0,0.0,0.0,0.012195,0.396313
4,1970-01-01 00:01:00,0.000088,0.0,0.047872,0.48,0.284916,1.0,0.0,0.0,0.012195,0.382488


In [11]:
df.tail()

,timestamp,index,machine_id,cpu_util_percent,mem_util_percent,mem_gps,mkpi,net_in,net_out,disk_io_percent,cpu_difference
22919102,1970-01-08 23:58:45,0.999903,0.0,0.238095,0.526316,0.463803,0.5,0.999358,1.0,0.068627,0.456
22919103,1970-01-08 23:59:00,0.999927,0.0,0.244898,0.526316,0.455449,0.5,0.999358,1.0,0.049020,0.496
22919104,1970-01-08 23:59:15,0.999952,0.0,0.278912,0.526316,0.311058,0.5,0.999358,1.0,0.049020,0.528
22919105,1970-01-08 23:59:30,0.999976,0.0,0.244898,0.526316,0.417263,0.5,1.000000,1.0,0.049020,0.448
22919106,1970-01-08 23:59:45,1.000000,0.0,0.278912,0.578947,0.352426,0.5,1.000000,1.0,0.049020,0.528


In [12]:
for col in df.columns:
    print(f"{col}: {df[col].nunique()} unique values")

timestamp: 46080 unique values
index: 22647331 unique values
machine_id: 1 unique values
cpu_util_percent: 2030241 unique values
mem_util_percent: 1740596 unique values
mem_gps: 587860 unique values
mkpi: 1148 unique values
net_in: 2889638 unique values
net_out: 2754166 unique values
disk_io_percent: 1489493 unique values
cpu_difference: 65864 unique values


In [13]:
df = df.drop(columns=["index", "machine_id"])
df.columns

Index(['timestamp', 'cpu_util_percent', 'mem_util_percent', 'mem_gps', 'mkpi',
       'net_in', 'net_out', 'disk_io_percent', 'cpu_difference'],
      dtype='object')

In [14]:
# Convert timestamp string to pandas datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')  # this safely parses strings like '1970-01-01 00:00:00'

# Extract time-based features
df['hour'] = df['timestamp'].dt.hour
df['minute'] = df['timestamp'].dt.minute
df['second'] = df['timestamp'].dt.second
df['day'] = df['timestamp'].dt.day
df['weekday'] = df['timestamp'].dt.weekday  # Monday = 0

# Drop original timestamp column
df = df.drop(columns=['timestamp'])

In [15]:
df = df[['hour', 'minute', 'second', 'day', 'weekday', 'mem_util_percent', 'mem_gps', 'mkpi', 'net_in', 'net_out', 'disk_io_percent', 'cpu_difference','cpu_util_percent' ]]

In [16]:
df.head()

,hour,minute,second,day,weekday,mem_util_percent,mem_gps,mkpi,net_in,net_out,disk_io_percent,cpu_difference,cpu_util_percent
0,0,0,0,1,3,0.48,0.284916,1.0,0.0,0.0,0.012195,NaN,0.010638
1,0,0,15,1,3,0.48,0.284916,1.0,0.0,0.0,0.012195,0.405530,0.031915
2,0,0,30,1,3,0.48,0.284916,1.0,0.0,0.0,0.012195,0.396313,0.042553
3,0,0,45,1,3,0.48,0.284916,1.0,0.0,0.0,0.012195,0.396313,0.053191
4,0,1,0,1,3,0.48,0.284916,1.0,0.0,0.0,0.012195,0.382488,0.047872


In [17]:
df.isna().sum()

hour                     0
minute                   0
second                   0
day                      0
weekday                  0
mem_util_percent         0
mem_gps             292731
mkpi                292731
net_in                   0
net_out                  0
disk_io_percent          0
cpu_difference         500
cpu_util_percent         0
dtype: int64

In [18]:
df = df.dropna(subset=['cpu_difference'])

df['mem_gps'] = df['mem_gps'].fillna(df['mem_gps'].median())
df['mkpi'] = df['mkpi'].fillna(df['mkpi'].median())


In [19]:
df.isna().sum()

hour                0
minute              0
second              0
day                 0
weekday             0
mem_util_percent    0
mem_gps             0
mkpi                0
net_in              0
net_out             0
disk_io_percent     0
cpu_difference      0
cpu_util_percent    0
dtype: int64

In [20]:
import os

# Step 1: Define split sizes
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Step 2: Compute indices for splitting
n = len(df)
train_end = int(n * train_ratio)
val_end = train_end + int(n * val_ratio)

# Step 3: Perform the split (time-ordered)
train_df = df.iloc[:train_end]
val_df = df.iloc[train_end:val_end]
test_df = df.iloc[val_end:]

# Step 4: Create directory to save splits
output_dir = "Dataset/Alibaba_random_500/"
os.makedirs(output_dir, exist_ok=True)

# Step 5: Save to CSV
train_df.to_csv(os.path.join(output_dir, "train.csv"), index=False)
val_df.to_csv(os.path.join(output_dir, "val.csv"), index=False)
test_df.to_csv(os.path.join(output_dir, "test.csv"), index=False)

print("✅ Train/Val/Test splits saved to:", output_dir)
print(f"Train: {train_df.shape}, Val: {val_df.shape}, Test: {test_df.shape}")


✅ Train/Val/Test splits saved to: Dataset/Alibaba_random_500/
Train: (16043024, 13), Val: (3437791, 13), Test: (3437792, 13)
